[View in Colaboratory](https://colab.research.google.com/github/vincentei/predict_power_prices/blob/master/power_prices_RNN.ipynb)

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import datetime as dt

from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,Activation,GRU,RNN,TimeDistributed,RepeatVector,SimpleRNN

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [0]:
needUpload = True

In [3]:
if needUpload == True:
  from google.colab import files

  uploaded = files.upload()

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

Saving make_prices_lstm2.csv to make_prices_lstm2.csv
User uploaded file "make_prices_lstm2.csv" with length 1117728 bytes


In [0]:
# import the data
df = pd.read_csv('make_prices_lstm2.csv',usecols=['price','year'])

In [29]:
df['price'] = df['price']/100
df.head(2)

,price,year
0,0.3243,2014
1,0.3249,2014


In [30]:
# calc number of days in df, reshape df into (numdays,24)
numdays = int(len(df)/24)
dataX = df['price'].values.reshape(numdays,24)
print("There are {} days in the dataset".format(numdays))

There are 1597 days in the dataset


In [31]:
# check
dataX[0:2]

array([[0.3243, 0.3249, 0.2843, 0.2763, 0.2599, 0.2447, 0.172 , 0.0996,
        0.0949, 0.2655, 0.276 , 0.2844, 0.2869, 0.2883, 0.2906, 0.294 ,
        0.3211, 0.3499, 0.4496, 0.3999, 0.3496, 0.3287, 0.2873, 0.2785],
       [0.2791, 0.26  , 0.0496, 0.0099, 0.1999, 0.2842, 0.2934, 0.3846,
        0.4335, 0.4694, 0.55  , 0.66  , 0.6409, 0.6293, 0.5943, 0.5244,
        0.55  , 0.7808, 0.667 , 0.5353, 0.5244, 0.4496, 0.3547, 0.3064]])

In [32]:
# dataY is the same but all shifted
dataY = dataX[1:]
dataY[0]

array([0.2791, 0.26  , 0.0496, 0.0099, 0.1999, 0.2842, 0.2934, 0.3846,
       0.4335, 0.4694, 0.55  , 0.66  , 0.6409, 0.6293, 0.5943, 0.5244,
       0.55  , 0.7808, 0.667 , 0.5353, 0.5244, 0.4496, 0.3547, 0.3064])

In [0]:
# remove one day from dataX and check shape
dataX = dataX[:-1]

In [34]:
print(dataX.shape)
print(dataY.shape)

(1596, 24)
(1596, 24)


In [0]:
# split in train and test
numtraindays = 3*365
trainX = dataX[:numtraindays]
testX = dataX[numtraindays:]

trainY = dataY[:numtraindays]
testY = dataY[numtraindays:]

In [36]:
# reshape X into [samples,timestep,features]
trainX = trainX.reshape(numtraindays,24,1)
trainY = trainY.reshape(numtraindays,24,1)
print(trainX.shape)
print(trainY.shape)

(1095, 24, 1)
(1095, 24, 1)


In [37]:
len(testX)

501

In [38]:
numtestdays = int(len(testX))
testX = testX.reshape(numtestdays,24,1)
testY = testY.reshape(numtestdays,24,1)
print(testX.shape)
print(testY.shape)

(501, 24, 1)
(501, 24, 1)


In [76]:
# define RNN configuration
np.random.seed(5)
n_neurons = 1
batch_size = 1
n_epoch = 8
# create RNN
model = Sequential()
model.add(SimpleRNN(n_neurons, input_shape=(24, 1), return_sequences=True,use_bias=True))
model.add(TimeDistributed(Dense(1,activation = 'linear',use_bias=True)))
model.compile(loss='mean_squared_error', optimizer='adam')
print(model.summary())
# train RNN
model.fit(trainX, trainY, epochs=n_epoch, batch_size=batch_size, verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_17 (SimpleRNN)    (None, 24, 1)             3         
_________________________________________________________________
time_distributed_15 (TimeDis (None, 24, 1)             2         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/8
 - 3s - loss: 0.0140
Epoch 2/8
 - 3s - loss: 0.0085
Epoch 3/8
 - 3s - loss: 0.0077
Epoch 4/8
 - 3s - loss: 0.0073
Epoch 5/8
 - 3s - loss: 0.0070
Epoch 6/8
 - 3s - loss: 0.0069
Epoch 7/8
 - 3s - loss: 0.0069
Epoch 8/8
 - 3s - loss: 0.0068


In [0]:
#for layer in model.layers:
 # print (layer.get_weights())


In [77]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
print(testPredict.shape)

(501, 24, 1)


In [78]:
# calc the mean absolute error
print(testY.shape)
a = testY.reshape(501*24,1)
b = testPredict.reshape(501*24,1)
mean_absolute_error(a, b)*100

(501, 24, 1)


6.632693573137521

In [53]:
# https://stackoverflow.com/questions/38294046/simple-recurrent-neural-network-input-shape
np.random.seed(1337)

sample_size = 256
x_seed = [1, 0, 0, 0, 0, 0]
y_seed = [1, 0.8, 0.6, 0, 0, 0]

x_train = np.array([[x_seed] * sample_size]).reshape(sample_size,len(x_seed),1)
y_train = np.array([[y_seed]*sample_size]).reshape(sample_size,len(y_seed),1)

model=Sequential()
model.add(SimpleRNN(input_dim  =  1, output_dim = 50, return_sequences = True))
model.add(TimeDistributed(Dense(output_dim = 1, activation  =  "sigmoid")))
model.compile(loss = "mse", optimizer = "rmsprop")
print(model.summary())
model.fit(x_train, y_train, nb_epoch = 10, batch_size = 32)

print(model.predict(np.array([[[1],[0],[0],[0],[0],[0]]])))
#[[[ 0.87810659]
#[ 0.80646527]
#[ 0.61600274]
#[ 0.01652312]
#[ 0.00930419]
#[ 0.01328572]]]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `SimpleRNN` call to the Keras 2 API: `SimpleRNN(return_sequences=True, input_shape=(None, 1), units=50)`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_9 (SimpleRNN)     (None, None, 50)          2600      
_________________________________________________________________
time_distributed_8 (TimeDist (None, None, 1)           51        
Total params: 2,651
Trainable params: 2,651
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
256/256 [==============================] - 0s 1ms/step - loss: 0.1391
Epoch 2/10
256/256 [==============================] - 0s 117us/step - loss: 0.0721
Epoch 3/10
256/256 [==============================] - 0s 138us/step - loss: 0.0454
Epoch 4/10
256/256 [==============================] - 0s 115us/step - loss: 0.0312
Epoch 5/10
256/256 [==============================] - 0s 115us/step - loss: 0.0228
Epoch 6/10
256/256 [==============================] - 0s 110us/step - loss: 0.0173
Epoch 7/10
256/256 [===============